<a href="https://colab.research.google.com/github/harshitadd/FedBased_X-RayForCoVID/blob/master/Fed_Covid_customdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 ##Importing Mini Batch Data 
import os 
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install pydicom
!pip install tensorflow_federated

In [0]:
#%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_federated as tff
import pydicom 
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Conv1D,Flatten


In [0]:
# NUM_EPOCHS = 2 
# SHUFFLE_BUFFER = 10
# BATCH_SIZE = 10 
# PREFETCH_BUFFER = 5 

In [0]:
dataset_paths = {
  'client_0': '/content/drive/My Drive/tmp/a.csv',
  'client_1': '/content/drive/My Drive/tmp/b.csv',
  'client_2': '/content/drive/My Drive/tmp/c.csv',
}

def create_tf_dataset_for_client(id):
   path = dataset_paths.get(id)
   print(path)
   if path is None:
     raise ValueError(f'No dataset for client {id}')
   with tf.Session() as sess:
     obj = tf.data.TextLineDataset(path) 
     print(obj.eval())
   
   return tf.data.TextLineDataset(path)



In [0]:
train_data = tff.simulation.ClientData.from_clients_and_fn(
  ['client_0','client_1','client_2'], create_tf_dataset_for_client)

/content/drive/My Drive/tmp/a.csv
<TextLineDatasetV2 shapes: (), types: tf.string>


In [0]:
def make_federated_data(client_data, client_ids):
  return [client_data.create_tf_dataset_for_client(x)
      for x in client_ids
  ]

# def make_federated_data(client_data, client_ids):
#     return [preprocess(client_data.create_tf_dataset_for_client(x))
#           for x in client_ids]

In [0]:
federated_train_data = make_federated_data(train_data, train_data.client_ids)

/content/drive/My Drive/tmp/a.csv
<TextLineDatasetV2 shapes: (), types: tf.string>
/content/drive/My Drive/tmp/b.csv
<TextLineDatasetV2 shapes: (), types: tf.string>
/content/drive/My Drive/tmp/c.csv
<TextLineDatasetV2 shapes: (), types: tf.string>


In [0]:
# def preprocess(dataset):

#   return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
#       BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [0]:
# federated_train_data = tf.convert_to_tensor(federated_train_data)
# with tf.Session() as sess:
#   federated_train_data.eval()

In [0]:
print(federated_train_data)

[<TextLineDatasetV2 shapes: (), types: tf.string>, <TextLineDatasetV2 shapes: (), types: tf.string>, <TextLineDatasetV2 shapes: (), types: tf.string>]


In [0]:
print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

Number of client datasets: 3
First dataset: <TextLineDatasetV2 shapes: (), types: tf.string>


In [0]:

# #tf.keras.backend.clear_session() # For easy reset of notebook state.

# model = tf.keras.Sequential()
# #Conv Net for training 
# model.add(Conv1D(64, kernel_size=6, activation='relu', input_shape=(1024,1024)))
# model.add(Conv1D(32, kernel_size=6, activation='relu'))
# model.add(Flatten())
# model.add(Dense(2, activation='softmax'))


In [0]:
# model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# model.summary()

In [0]:
import numpy as np
type(federated_train_data)
np.shape(federated_train_data)

(3,)

In [0]:
def create_keras_model():

  model = tf.keras.Sequential()
  #Conv Net for training 
  model.add(Conv1D(64, kernel_size=6, activation='relu', input_shape=(1024,1024)))
  model.add(Conv1D(32, kernel_size=6, activation='relu'))
  model.add(Flatten())
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  model.summary()
  return model

In [0]:
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(keras_model, input_spec=(2,1),loss='binary_crossentropy')

In [0]:
# iterative_process = tff.learning.build_federated_averaging_process(model_fn)

# state = iterative_process.initialize()

# NUM_ROUNDS = 11
# for round_num in range(2, NUM_ROUNDS):
#     state, metrics = iterative_process.next(state, federated_train_data)
#     print('round {:2d}, metrics={}'.format(round_num, metrics))

In [132]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1019, 64)          393280    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1014, 32)          12320     
_________________________________________________________________
flatten (Flatten)            (None, 32448)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 64898     
Total params: 470,498
Trainable params: 470,498
Non-trainable params: 0
_________________________________________________________________


ValueError: ignored